In [1]:
import sklearn
import numpy as np
from random import randrange
from keras import layers, models, optimizers, backend, metrics, callbacks
import codecs
from keras.preprocessing import text, sequence
from nltk.tokenize import word_tokenize
import random

import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline
plt.rcParams['figure.figsize'] = (15, 12) # set default size of plots

/opt/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import csv

Решим задачу классификации текстовых данных. В качестве датасета возьмём базу рецензий сайта IMDB, рецензии размечены на два класса: позитивные и негативные. Такая задача называется sentiment analysis

1. Считаем данные из CSV файла

In [3]:
texts = []
labels = []
with open('labeledTrainData.tsv', 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter='\t')
    _ = next(reader)
    for l in reader:
        texts.append(l[2])
        labels.append(l[1])

Убедимся, что данные считались корректно:

In [4]:
texts[0]

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally sta

В данных сохранилась HTML-разметка. Вероятно, её стоит убрать.

Теперь необходимо перевести текст в такое представление, с которым нам будет удобно работать. Существует модель [мешка слов](https://en.wikipedia.org/wiki/Bag-of-words_model), которая долгое время использовалась в классических методах. К сожалению, эта модель не учитывает семантическую информацию и векторы, присваиваемые словам, имеют большую размерность, что делает её не лучшим выбором для тренировки нейронной сети.

Мы будем пользоваться [word embeddings](https://en.wikipedia.org/wiki/Word_embedding), специальными векторами рассчитанными таким образом, чтобы учитывать сематническую информацию и при этом иметь небольшой размер. Подробно про рассчёт embeddings на примере Word2Vec можно прочесть в [википедии](https://en.wikipedia.org/wiki/Word2vec), [оригинальносй статье](https://arxiv.org/abs/1301.3781) или слайдах курса.

Существует несколько видов embedding'ов. Сначала воспользуйтесь [Glove](https://nlp.stanford.edu/pubs/glove.pdf).


Скачанный файл состоит из двух столбцов: слово и вектор, который ему соответсвует.
    
Слова упорядочены по частоте встречаемости в языке. Вам необходимо загрузить эти векторы в словарь.

Vocab это вспомогательный класс, помогающий работать с вокабуляром. Внутри находятся два словаря, в одном хранится соответсвие между словами и индексами в glove (порядковый номер слова), а в другом -- между индексами и словами.

In [6]:
class Vocab(object):
    w2i = None # Word to index
    i2w = None

Следующая функция загружает эмбеддинги с диска. Обратите внимание, что добавляются два специальных ветора: один (PAD) отвечает за отступ (состоит из нулей и имеет нулевой индекс), а другой (UNK) -- за неизвестное слово (заполните его средним значением). Дополните её!

In [9]:
np.mean([[1,2], [2,3], [3,1], [3,1]], axis=0)

array([2.25, 1.75])

In [12]:
def load_embeddings(filename, embedding_size=300):
    
    # список векторов
    embeddings_list = []
    # словарь слово-индекс
    vocabulary_mapping = {'<PAD>': 0} # занесём ключ соотетствующий отступу
    pad = np.zeros(embedding_size) # создадим вектор для PAD
    embeddings_list.append(pad)
    
    with codecs.open(filename, 'rb', 'utf-8') as glove_file:
        for line in glove_file:
            token, vector = line.strip().split() # возможно этот код будет работать не всегда, исправьте его
            # Впишите сюда свой код
            # Обратите внимание, что значения в словаре должны совпадать с индексами в списке 
            
            
    vocabulary_mapping['<UNK>'] = len(embeddings_list)
    unk = np.mean(embeddings_list[1:], axis=0) # считаем средний вектор
    embeddings_list.append(unk)
    
    embeddings = np.array(embeddings_list)
    
    # создаём вокабуляр
    vocab = Vocab()
    vocab.i2w = {v: k for k, v in vocabulary_mapping.items()}
    vocab.w2i = vocabulary_mapping
    
    print('loaded!')
    return embeddings, vocab

Загрузим векторы с диска

In [ ]:
embeddings, vocab = load_embeddings('путь до файла', 100)

Теперь напишем функцию, которая будет разбивать предложения на токены (слова), а затем каждому токену ставить в соответсвие индекс вектора. Ограничим марсимальный размер текста 128 словами (из соображений скорости вычислений). Данное ограничение сильно влияет на качество, если Вы хотите достичь лучших результатов, то необходимо использовать весь текст (и изменить архитектуру сети).

In [13]:
def sent_to_id_vec(sent, vocab, max_len=128, mode='tokenize'):
    sent = sent.lower()

    # два режима токенизации
    if mode == 'tokenize':
        tokens = word_tokenize(sent)
    elif mode == 'split':
        tokens = sent.split()
    else:
        raise Error(f'Unknown mode: {mode}')

    if max_len is not None and len(tokens) > max_len:
        tokens = tokens[:max_len]

    result = []

    for token in tokens:
        if token in vocab.w2i:
            result.append(vocab.w2i[token])
        else:
            result.append(vocab.w2i['<UNK>'])

    return result

Векторизуем наш датасет:

In [14]:
sequences = []
for t in texts:
    temp = sent_to_id_vec(t, vocab)
    sequences.append(temp)

Теперь в каждом объекте датасета находится не текст, а последовательность идентификаторов.

In [ ]:
sequences[0]

Проверим, что мы не ошиблись и выполним обратное преобразование для произвольного предложения.

In [ ]:
' '.join([vocab.i2w[i] for i in sequences[0]])

Предложения совпадают, значит мы сделали верные преобразования

Рекуррентные сети способны улавливать зависимости между словами и чувствительны к порядку. Однако, они склонны "забывать" хвосты длинных последовательностей.

Однако, наивная реализация RNN-ячейки не способна показать сколько-нибудь значимые результаты. Воспользуемся ячейкой специального вида, называющейся LSTM. Про LSTM можно прочесть [в блоге Криса Ола](http://colah.github.io/posts/2015-08-Understanding-LSTMs/) и слайдах лекций.



In [ ]:
embeddings.shape

In [17]:
def build_LSTM_classifier():
    # Точка входа в граф задаётся при помощи специальных тензоров типа Input
    # Первая координата соответсвует длине текста, так как тексты в датасете имеют разную длину
    # значение считается переменным
    text_input = layers.Input(shape=(None,), dtype='int32')
    
    # Создаём специальный слой для работы с embedding, 
    # Его функция -- заменять индентификатор вектором из Glove
    # Указываем trainable = False, чтобы векторы embedding'ов не изменялись в процессе обучения
    embedding_layer = layers.Embedding(input_dim = embeddings.shape[0], 
                                       output_dim = embeddings.shape[1], 
                                       weights=[embeddings],
                                       mask_zero=True,
                                       trainable = False)
    
    x = embedding_layer(text_input)
    
    # Создаём рекуррентную ячейку
    # Первый параметр отвечает за размер внутреннего состояния (памяти ячейки)
    # По умолчанию такой слой возвращает только последнее состояние (см. картинку),
    # Если мы хотим получить состояния на каждом шаге необходимо указать return_sequences = True
    x = layers.LSTM(256, recurrent_dropout=0.25)(x)
    
    # Полученный результат направляем в полносвязный слой, который будет осуществлять классификацию
    output = layers.Dense(1, activation='sigmoid')(x)
    
    model = models.Model(inputs=[text_input], outputs=[output], name = 'LSTM_classifier')
    
    
    #  Для оптимизации будем использовать Adam 
    adam = optimizers.Adam(lr=0.0001)
    
    #Перед испльзованием модель необходимо скомпилировать
    model.compile(adam, 'binary_crossentropy', metrics=['acc'])
    
    return model 

In [18]:
backend.clear_session()
model = build_LSTM_classifier()

Выведем информацию по модели

In [19]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, None, 100)         200       
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               365568    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 366,025
Trainable params: 365,825
Non-trainable params: 200
_________________________________________________________________


Разобьём датасет на три части

In [ ]:
def split_train_val(train_size = 0.6, val_size = 0.1, test_size = 0.3):
    boundary_train = int(len(sequences) * train_size)
    boundary_val = int(len(sequences) * (train_size + val_size))
    
    train_set = (sequences[:boundary_train], labels[:boundary_train])
    val_set = (sequences[boundary_train:boundary_val], labels[boundary_train:boundary_val])
    test_set = (sequences[boundary_val:], labels[boundary_val:])
    
    return train_set, val_set, test_set

In [ ]:
(x_train, y_train), (x_val, y_val), (x_test, y_tes) = split_train_val()

Почти всё готово, чтобы начать обучение. Но так, как все предлдожения разной длины мы не можем конвертировать x в тензор, нам необходимо выровнять длину. Для этого мы воспользуемся специальной функцией pad_sequences(), доступной в keras. Недостающие элементы будут заполнены специальным символом PAD

Но нам не хотелось бы увеличивать все элементы в датасете по размеру максимального. По этогому будем генерировать батчи сами и применять pad_sequences к ним независимо

In [ ]:
def generate_batches(x, y, batch_size=64):
    i = 0
    while True:
        i = i % len(x)
        yield sequence.pad_sequences(x[i:i+batch_size]), y[i:i+batch_size]
        i += batch_size

In [ ]:
train_generator = generate_batches(x_train, y_train)
val_generator = generate_batches(x_train, y_train)

Теперь обучим нашу модель

In [ ]:
cbs = [
    callbacks.ModelCheckpoint('models/LSTM/{epoch:02d}-{val_loss:.4f}.h5', save_best_only=True),
    callbacks.TensorBoard(log_dir='models/LSTM/'),
    callbacks.ReduceLROnPlateau(factor=0.1, patience=3, verbose=1, min_lr=0.00001, epsilon=0.01)]


model.fit_generator(generator= train_generator, 
                    validation_data = val_generator, 
                    validation_steps= 40,
                    steps_per_epoch=256,
                    epochs=20,
                    callbacks=cbs)

In [ ]:
model.evaluate(sequence.pad_sequences(x_test), y=y_tes, batch_size=64)

77% Неплохой результат, но можно лучше.

Проблема в том, что RNN забывают начало последовательности, а в нашем датасете все рецензии достаточно велики. Есть способ уменьшить влияние этого эффекта -- обучить два LSTM, один идёт от начала к концу предложения, а другой -- от конца к началу. Такой подход называется bidirectional

** Обучите Bidirectional LSTM **


** Поэксперементируйте с эмбеддингами **


** Поэксперементируйте с видом ячеек, попробуйте сделать стэк **


** Уберите ограничение на 128 слов, адаптируйте модель к возросшей длине  **

** Попробуйте реализовать attention (или успешно применить чей-то) **

** Вы вольны делать всё что угодно, чтобы достичь наилучших результатов **